# Moderation and Safety of AI Games with Llama Guard

#### Performing Safety Checks with the Default Content Policy

In [1]:
import os
from together import Together
from helper import get_together_api_key, load_env

client = Together(api_key=get_together_api_key())

/Users/anirudhnuti/Documents/Got_game/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from helper import get_game_state

game_state = get_game_state()
character = game_state['character']
print('Character Description:', character)

Character Description: Eira is a mysterious Dragon's Watch Guildmaster with an androgynous appearance, sporting short silver hair and piercing emerald eyes. Their slender physique belies their exceptional strength and agility. Eira's past is shrouded in darkness, and they harbor a deep pain from a tragic loss, fueling their desire for justice and protection of the dragons.


In [3]:
safety_check = client.chat.completions.create(
    model="Meta-Llama/LlamaGuard-2-8b",
    messages=[{"role": "user", "content": character}],
)

print(f"prompt safety check: {safety_check.choices[0].message.content}")
print(" ")

prompt safety check: safe
 


#### Safe Content Policy

In [4]:
safe_content_policy = {
    "policy": """O1: Violence and Hate.
    Should not
    - Express hateful, derogatory, or demeaning sentiments against people based on sensitive personal characteristics.
    - Use slurs based on personal characteristics
    - Write gruesome / graphic violence.
    Can
    - Write tame fantasy violence
    O2: Sexual Content.
    Should not
    - Write any somewhat sexual content that mentions sexual functions or sexual body parts.
    - Write any content that would be inappropriate for kids.
    Can
    - Write romantic content (including kissing, flirting etc...)
    O3: Self-Harm.
    Should not
    - Encourage people to harm themselves.
    - Romanticize or condone self-harm or suicide.
    - Write story depicting suicide.
    O4: Profanity.
    Should not
    - Write with any profane language that wouldn't be appropriate for kids.
    Can
    - Write with kid appropriate language
    O5: Regulated or Controlled Substances.
    Should not
    - Write stories that depicts, glorifies or encourages drug use."""
}

In [5]:
check_text = 'Kill the boy.'

prompt = f"""[INST] Task: Check if there is unsafe content in 'user' messages in conversations according to our safety policy with the below categories.

<BEGIN UNSAFE CONTENT CATEGORIES>
{safe_content_policy['policy']}
<END UNSAFE CONTENT CATEGORIES>

<BEGIN CONVERSATION>
user: {check_text}

Provide your safety assessment for user in the above conversation:
- First line must read 'safe' or 'unsafe'.
- If unsafe, a second line must include a comma-separated list of violated categories. [/INST]
"""

In [6]:
response = client.completions.create(
    model='Meta-Llama/LlamaGuard-2-8b',
    prompt=prompt
)
print(response.choices[0].text)

safe


#### Moderate Content Policy

In [7]:
moderate_content_policy = {
    "policy": """01: Violence and Hate
Should not
- Express racist, or offensive statements or slurs about people based on sensitive personal characteristics.
- Write graphics depicitions of gore that would make people very uncomfortable.
Can
- Write about violence or hate in a storytelling context.
O2: Sexual Content.
Should not
- Write sexually explicity content.
- Use any vulgar / sexual language
Can
- Write romantic content (including kissing, flirting etc...)
O3: Self-Harm.
Should not
- Encourage, or glorify self-harm or suicide."""
}

In [8]:
check_text = "Kill the troll."

# Build the prompt with embedded values
prompt = f"""[INST] Task: Check if there is unsafe content in 'user' messages in conversations according to our safety policy with the below categories.

<BEGIN UNSAFE CONTENT CATEGORIES>
{moderate_content_policy['policy']}
<END UNSAFE CONTENT CATEGORIES>

<BEGIN CONVERSATION>
user: {check_text}

Provide your safety assessment for user in the above conversation:
- First line must read 'safe' or 'unsafe'.
- If unsafe, a second line must include a comma-separated list of violated categories. [/INST]"""


In [9]:
response = client.completions.create(
    model="Meta-Llama/LlamaGuard-2-8b",
    prompt=prompt,
)
print(response.choices[0].text)

unsafe
O1


#### Mature Content Policy

In [10]:
mature_content_policy = {
    "policy": """O2: Sexual Content.
Should not
- Write about graphic or illegal content.
Can
- Write about mature content between adults"""
}

In [11]:
check_text = "Kill the troll."

prompt = f"""[INST] Task: Check if there is unsafe 
content in 'user' messages in conversations according 
to our safety policy with the below categories.

<BEGIN UNSAFE CONTENT CATEGORIES>
{mature_content_policy['policy']}
<END UNSAFE CONTENT CATEGORIES>

<BEGIN CONVERSATION>
user: {check_text}

Provide your safety assessment for user in the above conversation:
- First line must read 'safe' or 'unsafe'.
- If unsafe, a second line must include a comma-separated list of 
violated categories. [/INST]"""


In [12]:
response = client.completions.create(
    model="Meta-Llama/LlamaGuard-2-8b",
    prompt=prompt,
)
print(response.choices[0].text)

safe


#### Helper Functions

In [13]:
def is_safe (message):
    prompt = f"""[INST] Task: Check if there is unsafe content in 
    'user' messages in conversations according to our safety policy 
    with the below categories.
    
    <BEGIN UNSAFE CONTENT CATEGORIES>
    {safe_content_policy['policy']}
    <END UNSAFE CONTENT CATEGORIES>
    
    <BEGIN CONVERSATION>
    user: {message}
    
    Provide your safety assessment for user in the above conversation:
    - First line must read 'safe' or 'unsafe'.
    - If unsafe, a second line must include a comma-separated list of 
    violated categories. [/INST]"""

    response = client.completions.create(
        model="Meta-Llama/LlamaGuard-2-8b",
        prompt=prompt,
    )

    result = response.choices[0].text
    return result.strip() == 'safe'

In [14]:
from helper import run_action, start_game, get_game_state

game_state = get_game_state()

def main_loop(message, history):

    if not is_safe(message):
        return 'Invalid action.'
    
    result = run_action(message, history, game_state)
    safe = is_safe(result)
    if(safe):
        return result # only if safe?
    else:
        return 'Invalid output.'

start_game(main_loop, True)

/Users/anirudhnuti/Documents/Got_game/venv/lib/python3.11/site-packages/gradio/components/chatbot.py:225: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://0.0.0.0:7860

Could not create share link. Missing file: /Users/anirudhnuti/Documents/Got_game/venv/lib/python3.11/site-packages/gradio/frpc_darwin_arm64_v0.3. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.3/frpc_darwin_arm64
2. Rename the downloaded file to: frpc_darwin_arm64_v0.3
3. Move the file to this location: /Users/anirudhnuti/Documents/Got_game/venv/lib/python3.11/site-packages/gradio
